# Grain Boundary 3D FCC Metals (Copper)

## 0. Introduction

This notebook demonstrates how to create a grain boundary structure for a 3D FCC metal, Cu in particular. The grain boundary is created by combining two slabs with different orientations. 

Aiming to reproduce the structure from the publication:
> **Timofey Frolov, David L. Olmsted, Mark Asta & Yuri Mishin**
> "Structural phase transformations in metallic grain boundaries"
> Nature Communications, volume 4, Article number: 1899 (2013)
> DOI: [10.1038/ncomms2919](https://www.nature.com/articles/ncomms2919) 

Focusing on reproducing the structures from Figure 1. b:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_planar_grain_boundary_3d_fcc_metal/0-figure-from-manuscript.webp" alt="Figure 1." width="400"/>

## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
MATERIAL_NAME = "Cu"

# Parameters for Phase 1
PHASE_1_MILLER_INDICES = (3, 1, 0)
PHASE_1_THICKNESS = 4  # in atomic layers
PHASE_1_USE_ORTHOGONAL_C = True

# Parameters for Phase 2
PHASE_2_MILLER_INDICES = (-3, -1, 0)
PHASE_2_THICKNESS = 4  # in atomic layers
PHASE_2_USE_ORTHOGONAL_C = True

INTERPHASE_GAP = 0.0  # in Angstrom

# Maximum area for the superlattice search algorithm
MAX_AREA = 200  # in Angstrom^2


### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+gc41b0b8-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_grain_boundary.ipynb")


### 1.3. Get input material

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.build import MaterialWithBuildMetadata

material = MaterialWithBuildMetadata.create(Materials.get_by_name_first_match(MATERIAL_NAME))

### 1.4. Preview Material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([material], repetitions=[3, 3, 1], rotation="0x")

## 2. Create grain boundary

### 2.1. Create Phase 1 and Phase 2 Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.

### 2.2. Get possible terminations for the slabs

In [ ]:
from mat3ra.made.tools.build.slab.helpers import get_slab_terminations

phase_1_terminations = get_slab_terminations(material=material, miller_indices=PHASE_1_MILLER_INDICES)
phase_2_terminations = get_slab_terminations(material=material, miller_indices=PHASE_2_MILLER_INDICES)

print(f"Phase 1 Terminations:")
for idx, termination in enumerate(phase_1_terminations):
    print(f"    {idx}: {termination}")
print(f"Phase 2 Terminations:")
for idx, termination in enumerate(phase_2_terminations):
    print(f"    {idx}: {termination}")

### 2.3. Visualize slabs for all possible terminations

In [ ]:
from mat3ra.made.tools.build.slab.helpers import create_slab

phase_1_slabs = [create_slab(material, miller_indices=PHASE_1_MILLER_INDICES, termination=termination) for termination
                 in phase_1_terminations]
phase_2_slabs = [create_slab(material, miller_indices=PHASE_2_MILLER_INDICES, termination=termination) for termination
                 in phase_2_terminations]

visualize([{"material": slab, "title": slab.name} for slab in phase_1_slabs],
          repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material": slab, "title": slab.name} for slab in phase_2_slabs],
          repetitions=[3, 3, 1], rotation="-90x")

### 2.4. Create slabs with specified Miller indices and thickness

In [ ]:
# Create phase 1 slab
phase_1_slab = create_slab(
    crystal=material,
    termination=None,  # Use default termination
    miller_indices=PHASE_1_MILLER_INDICES,
    number_of_layers=PHASE_1_THICKNESS,
    vacuum=0.0,
    use_orthogonal_c=PHASE_1_USE_ORTHOGONAL_C
)

# Create phase 2 slab  
phase_2_slab = create_slab(
    crystal=material,
    termination=None,  # Use default termination
    miller_indices=PHASE_2_MILLER_INDICES,
    number_of_layers=PHASE_2_THICKNESS,
    vacuum=0.0,
    use_orthogonal_c=PHASE_2_USE_ORTHOGONAL_C
)

# Visualize the phase slabs
visualize([{"material": phase_1_slab, "title": "Phase 1 Slab"}, {"material": phase_2_slab, "title": "Phase 2 Slab"}],
          repetitions=[2, 2, 1], rotation="-90x")


### 2.5. Create Grain Boundary

In [ ]:
from mat3ra.made.tools.build.grain_boundary.helpers import create_grain_boundary_planar


grain_boundary = create_grain_boundary_planar(
    phase_1_material=phase_1_slab,
    phase_2_material=phase_2_slab,
    phase_1_miller_indices=PHASE_1_MILLER_INDICES,
    phase_2_miller_indices=PHASE_2_MILLER_INDICES,
    phase_1_thickness=PHASE_1_THICKNESS,
    phase_2_thickness=PHASE_2_THICKNESS,
    max_area=MAX_AREA,
    max_area_ratio_tol = 0.1,
    max_length_tol = 0.05,
    max_angle_tol = 0.02,
)

## 3. Visualize the Material

In [ ]:
visualize([grain_boundary], repetitions=[1, 1, 1], viewer="wave")

## 4. Save the final material

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(grain_boundary)
download_content_to_file(grain_boundary.to_json(),
                         f"{MATERIAL_NAME}-{PHASE_1_MILLER_INDICES}-{PHASE_2_MILLER_INDICES}_grain_boundary.json")